In [55]:
import h3
import requests
import pandas as pd

In [56]:
GEO='37.444572,-122.160309'
GEO2 = '37.44009132316512,-122.16042161897487'
api_key= 'Z1XB-w87ksYmy5gJnj4gfLPJUJRkiyyPDLEtyjnMYy0'
url_range = f'https://browse.search.hereapi.com/v1/browse?at={GEO2}&in=circle:{GEO2};r=700&limit=100&apiKey={api_key}'
print(url_range)

https://browse.search.hereapi.com/v1/browse?at=37.44009132316512,-122.16042161897487&in=circle:37.44009132316512,-122.16042161897487;r=700&limit=100&apiKey=Z1XB-w87ksYmy5gJnj4gfLPJUJRkiyyPDLEtyjnMYy0


In [57]:
resp = requests.get(url=url_range)
data= resp.json()

In [58]:
title_list = []
cat_id_list = []
cat_def_list = []
cat_name_list = []
lat_list = []
lng_list = []

count = 0
for i in data['items']:
    if 'categories' not in i:
        continue
    else:
        title_list.append(str(i['title']))
        cat_id_list.append(str(i['categories'][0]['id']))
        cat_name_list.append(str(i['categories'][0]['name']))
        lat_list.append(str(i['position']['lat']))
        lng_list.append(str(i['position']['lng']))
        count +=1
print('Places nearby: ' + str(count))

Places nearby: 100


In [59]:
dict = {'title': title_list, 'cat_id': cat_id_list, 'cat_name': cat_name_list, 'latitude': lat_list, 'longitude': lng_list} 
cat_df = pd.DataFrame(dict)

In [60]:
def make_cat_def(row):
    if row['cat_id'][:3] == '100':
        return 'Eat & Drink'
    elif row['cat_id'][:3] == '200':
        return 'Going Out-Entertainment'
    elif row['cat_id'][:3] == '300':
        return 'Sights and Museums'
    elif row['cat_id'][:3] == '400':
        return 'Transport'
    elif row['cat_id'][:3] == '500':
        return 'Accommodations'
    elif row['cat_id'][:3] == '600':
        return 'Shopping'
    elif row['cat_id'][:3] == '700':
        return 'Business and Services'
    elif row['cat_id'][:3] == '800':
        return 'Facilities'
    elif row['cat_id'][:3] == '900':
        return 'Areas and Buildings'

In [61]:
cat_df['cat_definition'] = cat_df.apply(lambda row : make_cat_def(row), axis=1)

In [62]:
cat_df.head()

,title,cat_id,cat_name,latitude,longitude,cat_definition
0,"Megan K. Palmisano, M.D",800-8000-0158,Medical Services/Clinics,37.44011,-122.16117,Facilities
1,"Noman Khan, MD - Sports and Spine Orthopedic S...",800-8000-0158,Medical Services/Clinics,37.44011,-122.16117,Facilities
2,"William D. Brubaker, MD",800-8000-0155,Family/General Practice Physicians,37.44011,-122.16117,Facilities
3,"Christina T Devine, MD",800-8000-0158,Medical Services/Clinics,37.44017,-122.16119,Facilities
4,"Robert C. Bocian, MD, Faaaai, PHD",800-8000-0155,Family/General Practice Physicians,37.43987,-122.16119,Facilities


In [63]:
def make_h3_code(row):
    return h3.geo_to_h3(
            lat=row['latitude'],
            lng=row['longitude'],
            resolution=8)

In [64]:
cat_df['latitude']=cat_df['latitude'].apply(pd.to_numeric)
cat_df['longitude']=cat_df['longitude'].apply(pd.to_numeric)

In [65]:
cat_df['h3_code'] = cat_df.apply(lambda row : make_h3_code(row), axis=1)

In [66]:
cat_df.head()

,title,cat_id,cat_name,latitude,longitude,cat_definition,h3_code
0,"Megan K. Palmisano, M.D",800-8000-0158,Medical Services/Clinics,37.44011,-122.16117,Facilities,8828347433fffff
1,"Noman Khan, MD - Sports and Spine Orthopedic S...",800-8000-0158,Medical Services/Clinics,37.44011,-122.16117,Facilities,8828347433fffff
2,"William D. Brubaker, MD",800-8000-0155,Family/General Practice Physicians,37.44011,-122.16117,Facilities,8828347433fffff
3,"Christina T Devine, MD",800-8000-0158,Medical Services/Clinics,37.44017,-122.16119,Facilities,8828347433fffff
4,"Robert C. Bocian, MD, Faaaai, PHD",800-8000-0155,Family/General Practice Physicians,37.43987,-122.16119,Facilities,8828347433fffff


In [67]:
cat_df.nunique()

title             100
cat_id             12
cat_name           12
latitude            7
longitude           6
cat_definition      3
h3_code             1
dtype: int64

In [68]:
h3.h3_to_geo('8828347433fffff')

(37.44009132316512, -122.16042161897487)